<a href="https://colab.research.google.com/github/vvoroby/semestr_4_numerical_methods/blob/main/%D0%9A%D1%83%D1%80%D1%81%D0%BE%D0%B2%D0%B0%D1%8F_%D0%BF%D0%BE_%D0%A7%D0%9C_%D0%A7%D0%B8%D1%81%D0%BB%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D1%80%D0%B5%D1%88%D0%B5%D0%BD%D0%B8%D1%8F_%D0%BF%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D1%8B_%D1%81%D0%BE%D0%B1%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D1%85_%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D0%B9_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Численные методы решения проблемы собственных значений.**

In [1]:
import math
import cmath
import numpy as np

## **1. Степенной метод (степенные итерации).**
Степенной метод предназначен для решения частичной проблемы собственных значений — нахождения доминирующих собственного значения и собственного вектора матрицы.

In [2]:
def power_method(B, tol, x): #принимает на вход матрицу и доп.приближение и нач. вектор
  A = B.copy()
  iter = 0
  x = x.dot(1/np.linalg.norm(x, ord = 2)) # текущее приближение к нормированному доминирующему собственному вектору
  lamda = 0
  lamda_next = 1
  while abs(lamda_next - lamda) > tol :
    y = A.dot(x) # направление вектора преобладания
    lamda = lamda_next 
    lamda_next = (y.dot(x))/np.linalg.norm(x, ord = 2) # текущее приближение к доминирующему собственному значению
    x = y.dot(1/np.linalg.norm(y, ord = 2))
    iter = iter + 1
  print ('Кол-во итераций в степенном методе: ', iter)
  return round(lamda_next, 7), x #возвращает собственное значение и вектор


In [3]:
# функция этого же метода для отслеживания приближения к собственному значению на некоторых итерациях

def power_method_iter(B, tol, x): #принимает на вход матрицу и доп.приближение и нач. вектор
  A = B.copy()
  iter = 0
  x = x.dot(1/np.linalg.norm(x, ord = 2)) # текущее приближение к нормированному доминирующему собственному вектору
  lamda = 0
  lamda_next = 1
  while abs(lamda_next - lamda) > tol :
    if iter == 1:
      print ('1 - ', round(lamda_next, 7))
    if iter == 3:
      print ('3 - ', round(lamda_next, 7))
    if iter == 5:
      print ('5 - ', round(lamda_next, 7))
    if iter%10 == 0:
      print (iter,' - ', round(lamda_next, 7))

    y = A.dot(x) # направление вектора преобладания
    lamda = lamda_next 
    lamda_next = (y.dot(x))/np.linalg.norm(x, ord = 2) # текущее приближение к доминирующему собственному значению
    x = y.dot(1/np.linalg.norm(y, ord = 2))
    iter = iter + 1
  print ('Кол-во итераций в степенном методе: ', iter)
  return round(lamda_next, 7), x #возвращает собственное значение и вектор

## **2. Обратные степенные итерации.**

In [4]:
def inverse_power_iterations(B, tol, x):  #принимает на вход матрицу и доп.приближение и нач. вектор
  A = B.copy()
  iter = 0
  x = x.dot(1/np.linalg.norm(x, ord = 2)) # текущее приближение к нормированному доминирующему собственному вектору
  lamda = 0
  lamda_next = 1
  while abs(lamda_next - lamda) > tol :
    y = np.linalg.solve(np.array(A), np.array(x))
    lamda = lamda_next 
    lamda_next = ((x.dot(y))/np.linalg.norm(y, ord = 2))/((y.dot(y))/np.linalg.norm(y, ord = 2)) # текущее приближение к доминирующему собственному значению
    x = y.dot(1/np.linalg.norm(y, ord = 2))
    iter = iter + 1
  print ('Кол-во итераций в обратном степенном методе: ', iter)
  return round(lamda_next, 7), x #возвращает собственное значение и вектор 

## **3. Базовый QR - алгоритм.**

In [177]:
def convergence(A):
    sum = 0
    el = np.diag(A, k=-1) 
    for i in range(0, len(el)-1):
      sum += abs(el[i]*el[i+1])
    return(sum)

def QR_algorithm(B, tol):   
  A = B.copy()
  iter = 0
  while convergence(A) > tol:  
    Q, R = np.linalg.qr(A)
    A = np.round(R.dot(Q),5)
    iter += 1
  print ('Кол-во итераций в базовом QR алгоритме: ', iter)
  return A 

## **3.1. Модифицированный QR-алгоритм со сдвигами**

In [178]:
# Модифицированный QR-алгоритм со сдвигами
def convergence(A):
    sum = 0
    el = np.diag(A, k=-1) #список наддиагональных векторов
    for i in range(0, len(el)-1):
      sum += abs(el[i]*el[i+1])
    return(sum)
    
def modified_QR_algorithm(B, tol):
  A = B.copy()
  iter = 0
  shift = 0 # сдвиг, далее равен приближенному собственному значению значению A 
  I = np.eye(len(A)) # единичная матрица размера A
  while convergence(A) > tol:  #условие сходимости  
    vI = I.dot(shift)
    Q, R = np.linalg.qr(A - vI)
    A = np.round((R.dot(Q) + vI),5)
    shift = A[0][0]
    iter += 1
  print ('Кол-во итераций в QR алгоритме со сдвигом: ', iter)
  return A

### *Примеры*

In [179]:
A = np.array([[1,-2,3],[4,5,-6],[-7,8,9]])
tol = 0.001 

print('QR-алгоритм: ')
print(QR_algorithm(A, tol))
print('QR-алгоритм со сдвигами: ') 
print(modified_QR_algorithm(A, tol))

QR-алгоритм: 
Кол-во итераций в базовом QR алгоритме:  9
[[ 2.76049e+00 -9.64246e+00 -4.95429e+00]
 [ 7.88805e+00  9.48116e+00 -3.68680e-01]
 [ 1.00000e-04 -0.00000e+00  2.75836e+00]]
QR-алгоритм со сдвигами: 
Кол-во итераций в QR алгоритме со сдвигом:  10
[[ 4.685320e+00 -5.602900e+00  4.026510e+00]
 [ 1.192791e+01  7.556190e+00 -2.908960e+00]
 [ 4.600000e-04 -8.000000e-05  2.758480e+00]]


## **4. Метод Якоби поиска собственных чисел.**

Идея метода Якоби состоит в том, чтобы подходящими преобразованиями подобия от шага к шагу уменьшать норму внедиагональной части матрицы. 

In [84]:
def frobenius_norm(A): # считает норму Фробениуса из внедиагональных элемнтов, используется для условия сходимости
  sum = 0
  for i in range(len(A)):
    for j in range(len(A)):
      if i!=j:
        sum += A[i][j]*A[i][j]
  return math.sqrt(sum)

def Elem(a): # поиск внедиагонального элемента (неравного нулю) максимального по модулю, т.к. так эффективнее 
  n = len(a)
  Max = 0.0
  for i in range(n-1):
    for j in range(i+1,n):
      if abs(a[i][j]) >= Max:
        Max = abs(a[i][j])
        k = i; l = j
  return Max,k,l
    
def jacobi_method(B, tol):
  A = B.copy()
  iter = 0
  n = len(A)
  vec = np.eye(n)
    
  while frobenius_norm(A) >= tol: # условие сходимости
    elem, i, j = Elem(A) 
    q = A[i][i] - A[j][j]
    if q == 0:
      c = (math.sqrt(2))/2
      s = (math.sqrt(2))/2
      print (c, s)
    else:
      p = 2*A[i][j]
      d = math.sqrt(p*p + q*q)
      r = abs(q)/(2*d)
      c = math.sqrt(0.5 + r)
      s = (math.sqrt(0.5 - r))*(np.sign(p*q))

    I = np.eye(n) # матрица поворота
    I[i][j] = -s
    I[j][i] = s
    I[i][i] = c
    I[j][j] = c
    #print ('I', I)

    A = np.round(np.transpose(I).dot(A).dot(I),2) 
    vec = vec.dot(I)
    iter += 1

  #print(A)
  print ('Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением максимального: ', iter)
  return np.diag(A), vec #возвращает собственные значения и вектора

In [109]:
def frobenius_norm(A): # считает норму Фробениуса из внедиагональных элемнтов, используется для условия сходимости
  sum = 0
  for i in range(len(A)):
    for j in range(len(A)):
      if i!=j:
        sum += A[i][j]*A[i][j]
  return math.sqrt(sum)

def Elem(a): # поиск внедиагонального элемента неравного нулю
  n = len(a)
  Max = 0.0
  for i in range(n-1):
    for j in range(i+1,n):
      if abs(a[i][j]) > 0.00001:
        Max = abs(a[i][j])
        k = i; l = j
  return Max,k,l
  
def jacobi_method_zero(B, tol):
  A = B.copy()
  iter = 0
  n = len(A)
  vec = np.eye(n)
    
  while frobenius_norm(A) >= tol: # условие сходимости
    elem, i, j = Elem(A) 
    q = A[i][i] - A[j][j]
    if q == 0:
      c = (math.sqrt(2))/2
      s = (math.sqrt(2))/2
      print (c, s)
    else:
      p = 2*A[i][j]
      d = math.sqrt(p*p + q*q)
      r = abs(q)/(2*d)
      c = math.sqrt(0.5 + r)
      s = (math.sqrt(0.5 - r))*(np.sign(p*q))

    I = np.eye(n) # матрица поворота
    I[i][j] = -s
    I[j][i] = s
    I[i][i] = c
    I[j][j] = c

    A = np.round(np.transpose(I).dot(A).dot(I),2) 
    vec = vec.dot(I)
    iter += 1

  #print(A)
  print ('Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением не нуля: ', iter)
  return np.diag(A), vec #возвращает собственные значения и вектора

### *Примеры*

In [ ]:
A = np.array([[1,2,5,4], [2,3,6,2],[5,6,4,1],[4,2,1,6]])# в функции пока нет проверки на симметричность
#A = np.array([[45, 27],[27, 45]])
tol = 0.001
B, v = jacobi_method(A, tol)
B_2, v_2 = jacobi_method_zero(A, tol)

Предварительно, матрицу A можно привести к трехдиагональному
виду с помощью метода Гивенса

## **5. Метод Якоби поиска сингулярных чисел.**

In [85]:
def search_singular_values(B, tol):
  def frobenius_norm(A): # считает норму Фробениуса из внедиагональных элемнтов, используется для условия сходимости
    sum = 0
    for i in range(len(A)):
      for j in range(len(A)):
        if i!=j:
          sum += A[i][j]*A[i][j]
    return math.sqrt(sum)

  def Elem(a): # поиск внедиагонального элемента (неравного нулю) максимального по модулю, т.к. так эффективнее 
    n = len(a)
    Max = 0.0
    for i in range(n-1):
      for j in range(i+1,n):
        if abs(a[i][j]) >= Max:
          Max = abs(a[i][j])
          k = i; l = j
    return Max,k,l

  G = B.copy()
  A = np.transpose(G).dot(G)
  iter = 0
  n = len(A)
  vec = np.eye(n)
    
  while frobenius_norm(A) >= tol: # условие сходимости
    elem, i, j = Elem(A) 
    q = A[i][i] - A[j][j]
    if q == 0:
      c = (math.sqrt(2))/2
      s = (math.sqrt(2))/2
      print (c, s)
    else:
      p = 2*A[i][j]
      d = math.sqrt(p*p + q*q)
      r = abs(q)/(2*d)
      c = math.sqrt(0.5 + r)
      s = (math.sqrt(0.5 - r))*(np.sign(p*q))

    I = np.eye(n) # матрица поворота
    I[i][j] = -s
    I[j][i] = s
    I[i][i] = c
    I[j][j] = c

    A = np.round(np.transpose(I).dot(A).dot(I),3)
    iter += 1
    vec = vec.dot(I)

  #print(A)
  print ('Кол-во итераций в методе Якоби для поиска сингулярных чисел: ', iter)
  singular = []
  for i in range(len(A)):
    for j in range(len(A)):
      if i == j:
        singular.append(cmath.sqrt(A[i][j]))

  return singular, vec

### *Примеры*

In [ ]:
# встроенный метод
u, s, vh = np.linalg.svd(A, full_matrices=False)

In [ ]:
m = np.array([[1,2],[-3,4]])

In [ ]:
QR_algorithm(m, 0.0001)

# **ПРИМЕРЫ**

### Пример 1
Готов.

In [87]:
def generate_wilkinson_matrix(n):
  m = np.zeros((n,n))
  el = n
  for i in range(n):
    for j in range(n):
      if i == j:
        m[i][j] = el
        el -= 1
      if i == j-1:
        m[i][j]= 20
  return m

In [88]:
wilkinson_matrix = generate_wilkinson_matrix(20)

In [89]:
lamda_1, v_1 = power_method(wilkinson_matrix, 0.0001, np.array([1]*len(wilkinson_matrix)))
lamda_2, v_2 = inverse_power_iterations(wilkinson_matrix, 0.0001, np.array([1]*len(wilkinson_matrix)))

Кол-во итераций в степенном методе:  178
Кол-во итераций в обратном степенном методе:  18


In [90]:
lamda_3 = QR_algorithm(wilkinson_matrix, 0.0001)
lamda_4 = modified_QR_algorithm(wilkinson_matrix, 0.0001)

Кол-во итераций в базовом QR алгоритме:  0
Кол-во итераций в QR алгоритме со сдвигом:  0


### Пример 2
Не работает метод Якоби.

In [91]:
def generate_hilbert_matrix(n):
  m = np.zeros((n,n))
  for i in range(n):
    for j in range(n):
      m[i][j] = 1/(i+j+1)
  return m

In [92]:
hilbert_matrix = generate_hilbert_matrix(20)

In [93]:
lamda_1, v_1 = power_method(hilbert_matrix, 0.0001, np.array([1]*len(hilbert_matrix)))
lamda_2, v_2 = inverse_power_iterations(hilbert_matrix, 0.0001, np.array([1]*len(hilbert_matrix)))

Кол-во итераций в степенном методе:  6
Кол-во итераций в обратном степенном методе:  2


In [94]:
lamda_3 = QR_algorithm(hilbert_matrix, 0.00001)
#lamda_4 = modified_QR_algorithm(hilbert_matrix, 0.01) очень долгие вычисления

Кол-во итераций в базовом QR алгоритме:  3


In [95]:
#lamda_5, v_5 = jacobi_method(hilbert_matrix, 0.01)

In [96]:
#lamda_6, v_6 = jacobi_method_zero(hilbert_matrix, 0.0001)

### Пример 3

In [97]:
m_3 = np.array([[-2,3,4],[5,7,2],[60,3,5]])
lamda, vec = power_method_iter(m_3, 0.0001, np.array([1]*len(m_3)))
print('Доминирующее собственное значение: ', lamda, '\nСоответсьвующий собственный вектор: ', vec)

0  -  1
1 -  29.0
3 -  27.8447651
5 -  23.8872789
10  -  18.2558089
20  -  19.1815624
30  -  19.230119
40  -  19.2326253
Кол-во итераций в степенном методе:  44
Доминирующее собственное значение:  19.23272 
Соответсьвующий собственный вектор:  [0.2125117  0.24165996 0.94680475]


In [98]:
m_3 = m_3 + 5*np.eye(len(m_3)) #делаем сдвиг
lamda, vec = power_method_iter(m_3, 0.0001, np.array([1]*len(m_3)))
print('Доминирующее собственное значение: ', lamda - 5, '\nСоответсьвующий собственный вектор: ', vec)

0  -  1
1 -  34.0
3 -  26.5643365
5 -  24.5717084
10  -  24.230511
Кол-во итераций в степенном методе:  15
Доминирующее собственное значение:  19.2327872 
Соответсьвующий собственный вектор:  [0.21251104 0.24165989 0.94680492]


### Пример 4
готово.

In [99]:
m_4 = np.array([[1,1,0],[0,1,0],[0,0,1/2]])
x = np.array([1,1,1]) # начальный вектор
m_4

array([[1. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 0.5]])

In [100]:
lamda_1, v_1 = power_method(m_4, 0.0001, x)
lamda_2, v_2 = inverse_power_iterations(m_4, 0.0001, x)

Кол-во итераций в степенном методе:  101
Кол-во итераций в обратном степенном методе:  10


In [101]:
lamda_3 = QR_algorithm(m_4, 0.0001)
lamda_4 = modified_QR_algorithm(m_4, 0.0001)

Кол-во итераций в базовом QR алгоритме:  0
Кол-во итераций в QR алгоритме со сдвигом:  0


### Пример 5
готово.

In [102]:
m_5 = np.array([[1,-2,3],[4,5,-6],[-7,8,9]])

In [103]:
lamda_2, v_2 = inverse_power_iterations(m_5, 0.0001, x)
lamda_1, v_1 = power_method(m_5, 0.0001, x)

Кол-во итераций в обратном степенном методе:  9
Кол-во итераций в степенном методе:  115365


In [104]:
lamda_3 = QR_algorithm(m_5, 0.0001)
lamda_4 = modified_QR_algorithm(m_5, 0.0001)

Кол-во итераций в базовом QR алгоритме:  6
Кол-во итераций в QR алгоритме со сдвигом:  8


### Пример 6
готово.

In [105]:
m_6 = np.array([[0,1],[1,0]])
x = np.array([1,1])

In [106]:
lamda_1, v_1 = power_method(m_6, 0.0001, x)
lamda_2, v_2 = inverse_power_iterations(m_6, 0.0001, x)

Кол-во итераций в степенном методе:  1
Кол-во итераций в обратном степенном методе:  1


In [107]:
lamda_3 = QR_algorithm(m_6, 0.0001)
lamda_4 = modified_QR_algorithm(m_6, 0.0001)

Кол-во итераций в базовом QR алгоритме:  0
Кол-во итераций в QR алгоритме со сдвигом:  0


In [110]:
lamda_5, v_5 = jacobi_method(m_6, 0.0001)
lamda_6, v_6 = jacobi_method_zero(m_6, 0.0001)

0.7071067811865476 0.7071067811865476
Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением максимального:  1
0.7071067811865476 0.7071067811865476
Кол-во итераций в методе Якоби для поиска собственных чисел c обнулением не нуля:  1


### Пример 7

In [111]:
m_001 = np.array([[6.66683, 3.33317, -1.66633],[3.33317, 6.66683, 1.66633], [-1.66633, 1.66633, 1.66733]]) #собственные числа 0.001, 5, 10
m_1 = np.array([[6.83333, 3.16667, -1.33333],[3.16667, 6.83333, 1.33333],[-1.33333, 1.33333, 2.33333]]) #собственные числа 1, 5, 10
x = np.array([1,1,1])

In [112]:
print(power_method(m_001, 0.0001, x))
print(power_method(m_1, 0.0001, x))

Кол-во итераций в степенном методе:  9
(9.9999873, array([7.06646209e-01, 7.07566904e-01, 4.60347545e-04]))
Кол-во итераций в степенном методе:  9
(9.9999873, array([7.06646209e-01, 7.07566904e-01, 4.60348016e-04]))


In [113]:
print(inverse_power_iterations(m_001, 0.0001, x))
print(inverse_power_iterations(m_1, 0.0001, x))

Кол-во итераций в обратном степенном методе:  2
(0.001, array([ 0.40824829, -0.40824826,  0.8164966 ]))
Кол-во итераций в обратном степенном методе:  5
(1.0000002, array([ 0.40812989, -0.40810539,  0.8166272 ]))


In [114]:
print(QR_algorithm(m_001, 0.0001))
print(QR_algorithm(m_1, 0.0001))

Кол-во итераций в базовом QR алгоритме:  1
[[ 9.29  1.75  0.  ]
 [ 1.75  5.71 -0.  ]
 [ 0.   -0.    0.  ]]
Кол-во итераций в базовом QR алгоритме:  4
[[9.99 0.26 0.  ]
 [0.26 5.01 0.  ]
 [0.   0.   1.01]]


### Метод Гивенса
Метод Гивенса приводит любую симметричную (несимметричную) матрицу к подобной ей трехдиагональной (почти треугольной) матрице и основан на подобных преобразованиях исходной матрицы с помощью матриц вращения.

In [221]:
def givens_method(m):
  A = m.copy()
  n = len(A)
  for i in range(n-1):
    for j in range(i, n):
      t = np.eye(n)
      c = (A[i-1][i])/(math.sqrt((A[i-1][i])**2+(A[i-1][j])**2))
      s = -(A[i-1][j])/(math.sqrt((A[i-1][i])**2+(A[i-1][j])**2))
      t[i][i] = c
      t[j][j] = c
      t[i][j] = s
      t[j][i] = -s
      A = np.round((np.transpose(t).dot(A)).dot(t),3)
  return A


In [205]:
m_1 = np.array([[1,-2,3],[4,5,-6],[-7,8,9]])
m_2 = givens_method(m_1)

In [206]:
QR_algorithm(m_1, 0.001)
QR_algorithm(m_2, 0.001)

Кол-во итераций в базовом QR алгоритме:  9
Кол-во итераций в базовом QR алгоритме:  4


array([[ 9.25174e+00,  6.87838e+00, -7.62000e-02],
       [-8.78808e+00,  3.28209e+00, -2.94151e+00],
       [ 1.50000e-04, -1.00000e-05,  7.76190e-01]])

In [207]:
modified_QR_algorithm(m_1, 0.001)
modified_QR_algorithm(m_2, 0.001)

Кол-во итераций в QR алгоритме со сдвигом:  10
Кол-во итераций в QR алгоритме со сдвигом:  17


array([[ 8.411330e+00,  1.011842e+01,  1.507130e+00],
       [-5.547980e+00,  4.122700e+00, -2.527220e+00],
       [-4.100000e-04,  1.200000e-04,  7.759700e-01]])

In [217]:
m_1 = np.array([[1,4,5,6,43,45,3],
              [-12,45,6,7,8,-90,6],
              [5,6,78,9,5,3,-2],
              [4,6,-78,9,0,3,52],
              [1,3,-43,1,1,0,0],
              [0,0,-4,34,-21,0,1],
              [0,6,7,9,0,1,2]])
m_2 = givens_method(m_1)

<ipython-input-216-1bfd2b765817>:7: RuntimeWarning: invalid value encountered in true_divide
  c = (A[i-1][i])/(math.sqrt((A[i-1][i])**2+(A[i-1][j])**2))
<ipython-input-216-1bfd2b765817>:8: RuntimeWarning: invalid value encountered in true_divide
  s = -(A[i-1][j])/(math.sqrt((A[i-1][i])**2+(A[i-1][j])**2))


In [215]:
m_2

array([[-25.47,  -9.26,  28.24, -21.69,   5.24,  -1.46,   6.41],
       [ 31.42,  44.95, -18.47, -11.22,   1.51, -20.54,   7.43],
       [ -7.25,   5.1 ,  28.21,   5.65,  -0.61,   1.57,  -3.36],
       [ 17.55,  10.73, -12.75,  29.56,   3.39,  -5.22,   5.52],
       [-12.09,   5.12, -12.63,   5.7 ,  10.21,   0.55,   1.5 ],
       [  7.21,  20.12,   7.78,   2.18,  -4.04,  -5.89,  -4.76],
       [  1.38,   6.5 ,   7.19, -13.67, -15.81,   8.43, -19.21]])

In [209]:
QR_algorithm(m_1,0.001)
QR_algorithm(m_2,0.001)

Кол-во итераций в базовом QR алгоритме:  68
Кол-во итераций в базовом QR алгоритме:  0


array([[nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan]])

### Анализ продуктивности Леонтьева

In [228]:
m_3 = np.array([[1,2,3,4],
                [2,4,5,6],
                [3,5,0,1],
                [4,6,1,-13]])

In [232]:
QR_algorithm(m_3,0.001)

Кол-во итераций в базовом QR алгоритме:  7


array([[-1.508512e+01,  3.689700e+00,  1.300000e-04,  0.000000e+00],
       [ 3.689700e+00,  1.017054e+01, -2.500000e-04,  0.000000e+00],
       [ 1.300000e-04, -2.500000e-04, -3.125640e+00,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  4.023000e-02]])

In [229]:
m_4 = givens_method(m_3)
np.round(m_4,1)

array([[-14.5,   4.4,   0. ,   0. ],
       [  4.4,   8.8,   1.5,   0. ],
       [  0. ,   1.5,  -2.8,   0. ],
       [  0. ,   0. ,   0. ,   0. ]])

In [230]:
np.round(QR_algorithm(m_4,0.001),1)

Кол-во итераций в базовом QR алгоритме:  6


array([[-15.3,   0.3,   0. ,   0. ],
       [  0.3,   9.8,   0. ,   0. ],
       [  0. ,   0. ,  -3. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ]])